2018/9/26 Updates

Changes
1. Columns converted to proper datatypes
2. Entry and Exit numbers tracked for each turntile
    a. Entry is complete, exit can be completed using the same steps
    b. This should be handled by a function. One has been created but needs some debugging
3. df.head() in each block to show the progression of the cleaning. This will need to be removed
4. Some operations were performed because of the guidance of the challenge. 
   This is not always what we need for our project goals. I have kept the unused code in markedown cells at the end,
   incase we do need it later

Qustions:
1. Should an i.d. column be created that combines C_A, Unit, SCP, and Station? 

Next steps:
2. Fix functions
1. Calculate daily entry/exits for each day

In [1]:
import pandas as pd
import re
import dateutil.parser

In [2]:
#load April - July, 2017 & 2018 MTA Turntile data
#time intensive block
df = pd.read_csv('/Users/GabeKlick/Downloads/turnstile_recordings_summer_2017_2018_clean.csv')
df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [3]:
#rename index column to be used later for dateframes merges
df.rename({'Unnamed: 0': 'Index'}, axis='columns', inplace= True)
df.head()

,Index,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [4]:
#exit column has a few misplaced chars preventing the column from being int values
#this block removes the chars and converts columns to int64

#apply regex statement to ensure only numbers in the column
def only_num(col):
    return re.findall('(\d)', col)
df.exits = df.exits.apply(lambda x: only_num(str(x)))

#the regex statment returns a list a numbers for each value
#join list values to single string items of numbers
df.exits = df.exits.apply(lambda x: ''.join(x))

#convert str to int64
df.exits = df.exits.apply(lambda x: int(x))

df.head()

,Index,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [5]:
#converts date column from str to datetime
#time intensive. Run, then take 5
df.date = df.date.apply(lambda x: dateutil.parser.parse(x))
df.head()

,Index,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,00:00:00,REGULAR,6157740,2085315
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,04:00:00,REGULAR,6157777,2085319
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,08:00:00,REGULAR,6157810,2085353
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,12:00:00,REGULAR,6157963,2085453
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,16:00:00,REGULAR,6158212,2085529


In [ ]:
'''
NOT WORKING

Creates a new series that groups turnstyles by station, id, and date, 
and find the difference in consecutive data points
stores this difference as a series
crease a temp df using difference and the main df's index column
'''
def entry_exit_calc(column, new_col_name):
    new_series = df.groupby(['station','scp','date']).column.apply(lambda x: x- x.shift(1))
    return pd.DataFrame({'Index': df.Index, new_col_name: new_series})
    

In [ ]:
#merge main and temp df, see block above
#NOT WORKING
df = pd.merge(df, entry_exit_calc('entries', 'Entries'))

In [6]:
#This should be a function. See above
#time intensive. Run, then take 5

#Groups turnstiles by station, turnstyle id, and date
#within each group find the difference between the consecutive entry points 
#this difference is stored as a series
df.test = df.groupby(['station','scp','date']).entries.apply(lambda x: x - x.shift(1))

#create a temp df that uses the same index column as the main df
df_entry = pd.DataFrame({'Index': df['Index'], 'Entries': df.test})

#merge the main and temp df on the index column
df = pd.merge(df, df_entry, how = 'outer', on = 'Index')

df.head(10)

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,Index,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,Entries
0,0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,00:00:00,REGULAR,6157740,2085315,NaN
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,04:00:00,REGULAR,6157777,2085319,37.0
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,08:00:00,REGULAR,6157810,2085353,33.0
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,12:00:00,REGULAR,6157963,2085453,153.0
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,16:00:00,REGULAR,6158212,2085529,249.0
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,20:00:00,REGULAR,6158521,2085589,309.0
6,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-30,00:00:00,REGULAR,6158637,2085612,NaN
7,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-30,04:00:00,REGULAR,6158654,2085617,17.0
8,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-30,08:00:00,REGULAR,6158661,2085641,7.0
9,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-30,12:00:00,REGULAR,6158703,2085730,42.0


Unused Code Below

#Entry and Exits data recorded as a 'snapshot' every 4 hours
#Adding columns tracking passanger entery and exit evert 4 hours
df = df.assign(entry_count = df['entries'] - df['entries'].shift(1))
df = df.assign(exit_count = df['exits'] - df['exits'].shift(1))

This block can be used for the challenge but not neccessary for our goals for the project
returns values in the formate requested by the challages 

#This blocks combines the date and time columns so they may later be converted to a datetime type

#combines columns, separated by a space, into the pre-exisiting column 'date'
df.date = df.date + ' '+ df.time

#rename to showcase the new time feature in the column
df.rename({'date': 'date_time'}, axis = 'columns', inplace = True)

#removes the now redundant time column
df.drop(columns = ['time'], inplace= True)

#converting date_time from string to datetime
df.date_time = df.date_time.apply(lambda x: dateutil.parser.parse(x))


Again, may be used for the challenge, but not necessary for the project

#create a new column that combines the date and time columns, separated by a single space
df['date_time'] = df.date + ' '+ df.time
#Warning: time intensive block, run then take 5
#converting date_time from string to datetime
df.date_time = df.date_time.apply(lambda x: dateutil.parser.parse(x))
df.head()
#currently returning datetime in same formate it is in: mm/dd/yyyy hh:mm:ss
#different than the challenge scope, but seems 